In [ ]:
# Install necessary libraries
!pip install music21 keras tensorflow numpy


In [ ]:
import os
import numpy as np
from music21 import converter, instrument, note, chord, stream
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [ ]:
# Step 1: Preprocess MIDI Dataset
def preprocess_midi(dataset_path, save_path=None):
    notes = []
    for file in os.listdir(dataset_path):
        if file.endswith('.mid'):
            try:
                midi = converter.parse(os.path.join(dataset_path, file))
                parts = instrument.partitionByInstrument(midi)

                if parts:  # If the file has instrument parts
                    for part in parts.parts:
                        if 'Piano' in str(part):
                            notes_to_parse = part.recurse()
                            break
                else:
                    notes_to_parse = midi.flat.notes

                for element in notes_to_parse:
                    if isinstance(element, note.Note):
                        notes.append(str(element.pitch))
                    elif isinstance(element, chord.Chord):
                        notes.append('.'.join(str(n) for n in element.normalOrder))
            except Exception as e:
                print(f"Error processing {file}: {e}")

    # Save notes to file for future use
    if save_path:
        np.save(save_path, notes)
        print(f"Preprocessed notes saved to {save_path}")

    return notes


# Check if preprocessed notes already exist
def load_or_preprocess_notes(dataset_path, save_path):
    if os.path.exists(save_path):
        notes = np.load(save_path, allow_pickle=True)
        print(f"Loaded preprocessed notes from {save_path}")
    else:
        notes = preprocess_midi(dataset_path, save_path)
    return notes


In [ ]:
def transpose_notes(notes, semitones):
    transposed = []
    for note_str in notes:
        try:
            if '.' in note_str or note_str.isdigit():  # Chord
                chord_notes = [int(n) + semitones for n in note_str.split('.')]
                transposed.append('.'.join(map(str, chord_notes)))
            else:  # Single note
                n = note.Note(note_str)
                n.transpose(semitones, inPlace=True)
                transposed.append(str(n.pitch))
        except Exception as e:
            print(f"Error transposing note {note_str}: {e}")
            continue
    return transposed


In [ ]:
# Step 3: Prepare Sequences for Training
def prepare_sequences(notes, sequence_length):
    pitch_names = sorted(set(notes))
    n_vocab = len(pitch_names)

    # Map notes to integers
    note_to_int = {note: num for num, note in enumerate(pitch_names)}
    int_to_note = {num: note for note, num in note_to_int.items()}

    # Create input and output sequences
    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[n] for n in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape input and normalize
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)

    # One-hot encode output
    network_output = to_categorical(network_output, num_classes=n_vocab)

    return network_input, network_output, n_vocab, int_to_note

In [ ]:
# Step 4: Build the LSTM-based Model
def build_model(n_vocab, sequence_length):
    model = Sequential()
    model.add(Input(shape=(sequence_length, 1)))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001))
    return model


In [ ]:
# Step 5: Save the Last Epoch Callback
class SaveLastEpochCallback(tf.keras.callbacks.Callback):
    def __init__(self, save_path):
        self.save_path = save_path

    def on_epoch_end(self, epoch, logs=None):
        with open(self.save_path, 'w') as f:
            f.write(str(epoch + 1))  # Save the next epoch


def get_last_epoch(epoch_file_path):
    if os.path.exists(epoch_file_path):
        with open(epoch_file_path, 'r') as f:
            return int(f.read().strip())
    return 0

In [ ]:
# Step 6: Train the Model with Checkpoint to Save the Model after Each Epoch
def train_with_checkpoint(model, network_input, network_output, epochs, batch_size, validation_split=0.1, initial_epoch=0):
    checkpoint = ModelCheckpoint(
        "best_model.keras",
        monitor="loss",
        verbose=1,
        save_best_only=True,
        mode="min"
    )

    save_last_epoch = SaveLastEpochCallback(epoch_file_path)

    model.fit(
        network_input,
        network_output,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=validation_split,
        callbacks=[checkpoint, save_last_epoch],
        initial_epoch=initial_epoch
    )

In [ ]:
# Step 7: Generate Music with Temperature Sampling
def generate_advanced_music(model, network_input, int_to_note, n_vocab, sequence_length, num_notes, temperature=0.8):
    start = np.random.randint(0, len(network_input) - 1)
    pattern = network_input[start]
    prediction_output = []

    for note_index in range(num_notes):
        prediction_input = np.reshape(pattern, (1, sequence_length, 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)
        prediction = np.log(prediction + 1e-9) / temperature
        prediction = np.exp(prediction) / np.sum(np.exp(prediction))
        index = np.random.choice(range(len(prediction[0])), p=prediction[0])

        result = int_to_note[index]
        prediction_output.append(result)

        pattern = np.append(pattern, index)
        pattern = pattern[1:]

    return prediction_output


In [ ]:
def create_midi(prediction_output, output_file):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        if pattern == "-" or pattern == "" or pattern.startswith("-"):  # Handle invalid notes/chords
            print(f"Skipping invalid pattern: {pattern}")
            continue

        if ('.' in pattern) or pattern.isdigit():  # Chord
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                try:
                    note_obj = note.Note(int(current_note))
                    note_obj.storedInstrument = instrument.Piano()
                    notes.append(note_obj)
                except Exception as e:
                    print(f"Error processing chord note {current_note}: {e}")
                    continue
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:  # Note
            try:
                note_obj = note.Note(pattern)
                note_obj.offset = offset
                note_obj.storedInstrument = instrument.Piano()
                output_notes.append(note_obj)
            except Exception as e:
                print(f"Error processing note {pattern}: {e}")
                continue

        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=output_file)


In [ ]:
# Main Execution
if __name__ == "__main__":
    dataset_path = "/content/drive/MyDrive/midi_dataset"
    notes_file_path = "/content/drive/MyDrive/processed_notes.npy"
    model_file_path = "/content/drive/MyDrive/best_model.keras"
    epoch_file_path = "/content/drive/MyDrive/last_epoch.txt"

    # Load or preprocess the dataset
    notes = load_or_preprocess_notes(dataset_path, notes_file_path)

    # Convert notes to list if it's a numpy array
    if isinstance(notes, np.ndarray):
        notes = notes.tolist()

    # Optional: Augment data
    augmented_notes = notes[:]
    for semitones in [-2, -1, 1, 2]:
        augmented_notes.extend(transpose_notes(notes, semitones))

    sequence_length = 100
    network_input, network_output, n_vocab, int_to_note = prepare_sequences(augmented_notes, sequence_length)

    # Load the model
    model = load_model(model_file_path)

    # Generate new music (No training, directly generating)
    prediction_output = generate_advanced_music(
        model, network_input, int_to_note, n_vocab, sequence_length, num_notes=500
    )

    # Create and save the MIDI file
    output_midi_path = "/content/drive/MyDrive/music.mid"
    create_midi(prediction_output, output_midi_path)
    print(f"MIDI file generated: {output_midi_path}")


Loaded preprocessed notes from /content/drive/MyDrive/processed_notes.npy
Skipping invalid pattern: -2.3
Skipping invalid pattern: -2.2.5
Skipping invalid pattern: -1.3
Skipping invalid pattern: -2.1
Skipping invalid pattern: -1
MIDI file generated: /content/drive/MyDrive/music.mid
